In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# !pip install --upgrade datasets

In [6]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

In [3]:
# import torch
# torch.ones(1,2).cuda()

In [4]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib
import random
import time

random.seed(51)
np.random.seed(24)
torch.manual_seed(77)

K = 2
STITCH_BATCH_SIZE = 32 # todo study the effect
MAX_DEPTH = 16
THRESOULD = 0.9
TOTAL_THRESOULD = 0.9

RESULT_NAME = f"{int(time.time())}_result_plant_CKA_BS_{STITCH_BATCH_SIZE}_MD_{MAX_DEPTH}_T_{THRESOULD}_TT_{TOTAL_THRESOULD}_K_{K}"

EVAL_BATCH_SIZE = 64

from stitchnet.stitchonnx.utils import load_hf_train_val_dset

dataset_train, dataset_val = load_hf_train_val_dset('sampath017/plants')

dl_score = load_dl(dataset_train, STITCH_BATCH_SIZE)
data_score,t = next(iter(dl_score))
data_score = data_score.numpy()

Using custom data configuration sampath017--plants-b46ce6bc289739b0
Found cached dataset imagefolder (/home/jupyter-steerapi/.cache/huggingface/datasets/sampath017___imagefolder/sampath017--plants-b46ce6bc289739b0/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/sampath017___imagefolder/sampath017--plants-b46ce6bc289739b0/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f/cache-42a0da81642841ef.arrow and /home/jupyter-steerapi/.cache/huggingface/datasets/sampath017___imagefolder/sampath017--plants-b46ce6bc289739b0/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f/cache-ff7aeed0fe4ad290.arrow
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Loading cached processed dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/sampath017___imagefolder/sampath017--plants-b46ce6bc289739b0/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f/cache-0c31efc05ee7bb41.arrow
Loading cached processed dataset at /home/jupyter-steerapi/.cache/huggingface/datasets/sampath017___imagefol

In [5]:
# range(1)
k = 0
if os.path.exists(f'./_results/{RESULT_NAME}.txt'):
    with open(f'./_results/{RESULT_NAME}.txt', 'r') as f:
        k = len(f.read().split('\n'))

In [6]:
from tqdm import tqdm
from stitchnet.stitchonnx.report import Report, ReportPlants

scoreMapper = ScoreMapper(nets, data_score, scoring_method='CKA')
with ReportPlants(EVAL_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'a') as report:
    # for _ in tqdm(range(50)):
    generator = generate_networks(nets, scoreMapper, data_score, 
                          threshold=THRESOULD, totalThreshold=TOTAL_THRESOULD, 
                          maxDepth=MAX_DEPTH, sample=False, K=K)
    for i,(s,net) in enumerate(generator):
        try:
            netname = f"_results/{RESULT_NAME}/net{k:03}"
            report.evaluate(nets, net, netname, s, dataset_val, dataset_train)
            net.save(netname)
            k += 1
        except Exception as e:
            print('ERROR', e)
            pass

current depth: 1
potential next fragments: 2
potential next fragments after thresholding of 0.9: 1 ['1.0']
totalscore 1.0
diff sampled tensor(0.)
epoch 0 loss 8.988104149081499e-11 torch.Size([23328, 64]) torch.Size([23328, 64])
current depth: 2
potential next fragments: 2
potential next fragments after thresholding of 0.9: 1 ['1.0']
totalscore 1.0
diff sampled tensor(0.0026)
epoch 0 loss 2.2259588989142614e-07 torch.Size([5408, 192]) torch.Size([5408, 192])
current depth: 3
potential next fragments: 2
potential next fragments after thresholding of 0.9: 1 ['1.0']
totalscore 0.9999998807907104
diff sampled tensor(0.0718)
epoch 0 loss 9.68740077843592e-06 torch.Size([5408, 384]) torch.Size([5408, 384])
current depth: 4
potential next fragments: 2
potential next fragments after thresholding of 0.9: 1 ['1.0']
totalscore 0.9999997019767974
diff sampled tensor(0.1415)
epoch 0 loss 2.165955144184581e-05 torch.Size([5408, 256]) torch.Size([5408, 256])
current depth: 5
potential next fragments:

100%|██████████| 3/3 [00:01<00:00,  2.01it/s]


(175, 1000) (175,)


100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


accuracy 0.9318181818181818
saving to _results/1689063880_result_plant_CKA_BS_32_MD_16_T_0.9_TT_0.9_K_2/net000
totalscore 0.981082862292801
epoch 0 loss 0.0 torch.Size([32, 4096]) torch.Size([32, 4096])
current depth: 8
potential next fragments: 2
potential next fragments after thresholding of 0.9: 2 ['1.0', '0.98']
totalscore 0.981082979246992
epoch 0 loss 0.0 torch.Size([32, 4096]) torch.Size([32, 4096])


...

accuracy 0.9545454545454546
saving to _results/1689063880_result_plant_CKA_BS_32_MD_16_T_0.9_TT_0.9_K_2/net013
totalscore 0.9109280917815588
epoch 0 loss 0.0 torch.Size([32, 2048]) torch.Size([32, 4096])


100%|██████████| 3/3 [00:01<00:00,  1.62it/s]


(175, 1000) (175,)


100%|██████████| 1/1 [00:01<00:00,  1.24s/it]


accuracy 0.9772727272727273
saving to _results/1689063880_result_plant_CKA_BS_32_MD_16_T_0.9_TT_0.9_K_2/net014
current depth: 1
potential next fragments: 2
potential next fragments after thresholding of 0.9: 1 ['1.0']
totalscore 1.0
diff sampled tensor(0.)
epoch 0 loss 1.178479648705879e-06 torch.Size([1605632, 64]) torch.Size([1605632, 64])
current depth: 2
potential next fragments: 2
potential next fragments after thresholding of 0.9: 1 ['1.0']
totalscore 1.0
diff sampled tensor(0.9110)
epoch 0 loss 2.1059806472317643e-06 torch.Size([401408, 64]) torch.Size([401408, 64])
current depth: 3
potential next fragments: 2
potential next fragments after thresholding of 0.9: 1 ['1.0']
totalscore 1.0000001192092896
diff sampled tensor(2.2831)
epoch 0 loss 5.544479561148346e-06 torch.Size([401408, 128]) torch.Size([401408, 128])
current depth: 4
potential next fragments: 2
potential next fragments after thresholding of 0.9: 1 ['1.0']
totalscore 1.0000001192092896
diff sampled tensor(2.4594)
epo

In [7]:
# from stitchnet.stitchonnx.utils import accuracy_score_net_plants

In [8]:
# for x,t in dataset_val:
#     print(t)

In [9]:
# dataset_val[0]

In [10]:
# accuracy_score_net_plants(net, dataset_val, dataset_train)

In [11]:
# from sklearn.neighbors import KNeighborsClassifier
# knn = KNeighborsClassifier(n_neighbors=1)
# knn.fit(np.ones([100,10]), np.ones([100,1]))


In [12]:
# y = knn.predict(np.ones([1,10]))
# y

In [13]:
# dataset_val

In [14]:
print('DONE')

DONE
